In [116]:
!pip install -r requirements.txt

In [117]:
import psycopg2                                         # psycopg2 for connecting to the database   
from psycopg2 import Error                              # Error handling for psycopg2                        

In [118]:
# Setting the database and table name before preprocessing 

database_name = "cheque_processing_system"
drawee_bank_table = "Drawee_bank"
beneficiary_bank_table = "Beneficiary_bank"

In [119]:
# PostgreSQL connection credentials

db_params = {
    "user" :"postgres",         # username
    "password" : "1234",        # password
    "host" : "localhost",       # host
    "port" : "5432"             # port
}

In [120]:
database = False
try:
    connection = psycopg2.connect(**db_params) # type: ignore
    cursor = connection.cursor()
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{database_name}';")
    result = cursor.fetchone()
    if result:
        print(f"{database_name} already exists!")
    else:
        print(f"{database_name} does not exist, creating one")
        database = True
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    exit()
finally:
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

cheque_processing_system does not exist, creating one
PostgreSQL connection is closed


In [121]:
if database == True:
    try:
        connection = psycopg2.connect(**db_params)  # type: ignore
        connection.autocommit = True
        cursor = connection.cursor()
        cursor.execute(f"CREATE DATABASE {database_name}")
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
        exit()
    finally:
        cursor.close()
        connection.commit()
        connection.close()
        print("Database created successfully")
        print("PostgreSQL connection is closed")
else:
    print(f"{database_name} already exists")

db_params = {
    "user": "postgres",
    "password": "1234",
    "host": "localhost",
    "port": "5432",
    "database": database_name
}

Database created successfully
PostgreSQL connection is closed


In [122]:
table = False
def check_table(table_name):
    table = False
    try:
        connection = psycopg2.connect(**db_params) # type: ignore
        cursor = connection.cursor()
        cursor.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = '{table_name}');")
        result = cursor.fetchone()[0]
        if result:
            print(f"Table '{table_name}' exists.")
        else:
            table = True
            print(f"Table '{table_name}' does not exist, creating one")
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
        exit()
    finally:
        cursor.close()
        connection.commit()
        connection.close()
        print("PostgreSQL connection is closed")
    return table

In [123]:
def create_table(table_name):
    create_table_sql = f"""
    CREATE TABLE {table_name} (
        id SERIAL PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        account_number VARCHAR(20) NOT NULL UNIQUE,
        balance NUMERIC(12, 2) NOT NULL
    );
    """
    table = check_table(table_name)
    if table == True:
        try:
            connection = psycopg2.connect(**db_params)  # type: ignore
        except (Exception, Error) as error:
            print("Error while connecting to PostgreSQL", error)
            exit()
        else:
            cursor = connection.cursor()
            cursor.execute(create_table_sql)
            print("Table created successfully")
        finally:
            cursor.close()
            connection.commit()
            connection.close()
            print("PostgreSQL connection is closed")
    else:
        print(f"Table '{table_name}' already exists")

In [124]:
def insert_data_onto_table (table_name, data):
    
    columns = ['name', 'account_number', 'balance']
    placeholders = ', '.join(['%s'] * len(columns))
    columns_str = ', '.join(columns)
    
    insert_sql = f"""
        INSERT INTO {table_name} ({columns_str})
        VALUES ({placeholders})
    """
    
    try:
        connection = psycopg2.connect(**db_params)  # type: ignore
        cursor = connection.cursor()
        
        cursor.execute(f"select count(*) from {table_name};")
        if cursor.fetchone()[0] == 0:
            cursor.executemany(insert_sql, data)
            connection.commit()
            print("Data inserted successfully into the {table_name} table.")
        else:
            print(f"Data already exists in the {table_name} table.")
    except (Exception, Error) as error:
        print("Error inserting data into {table_name} table:", error)
    finally:
        cursor.close()
        connection.commit()
        connection.close()
        print("PostgreSQL connection is closed.")

In [125]:
drawer_data = [
    ('Alice Johnson', 'ACC123456789', 1200.50),
    ('Bob Smith', 'ACC987654321', 2450.75),
    ('Charlie Brown', 'ACC555666777', 3600.00),
    ('Diana Prince', 'ACC111222333', 750.30),
    ('Ethan Hunt', 'ACC444555666', 1800.00),
    ('Frank Castle', 'ACC888999000', 2100.20),
    ('Grace Hopper', 'ACC777888999', 5400.40),
    ('Hank Pym', 'ACC112233445', 3900.00),
    ('Irene Adler', 'ACC998877665', 500.00),
    ('James Bond', 'ACC007007007', 7000.70),
    ('Karen Page', 'ACC123789456', 3100.90),
    ('Luke Cage', 'ACC654321987', 850.00),
    ('Matt Murdock', 'ACC147258369', 6700.10),
    ('Nancy Drew', 'ACC369258147', 4200.80),
    ('Oliver Queen', 'ACC741852963', 5600.25),
    ('Pam Beesly', 'ACC963852741', 950.75),
    ('Quinn Mallory', 'ACC258369147', 2300.60),
    ('Rachel Green', 'ACC456123789', 670.20),
    ('Steve Rogers', 'ACC963741852', 8900.50),
    ('Tony Stark', 'ACC852963741', 100000.00)
]

In [126]:
beneficiary_data = [
    ('John Doe', 'BEN123456789', 1500.75),
    ('Jane Smith', 'BEN987654321', 3200.50),
    ('Michael Johnson', 'BEN555666777', 4500.25),
    ('Emily Davis', 'BEN111222333', 1800.60),
    ('William Brown', 'BEN444555666', 2100.10),
    ('Olivia Wilson', 'BEN888999000', 3000.00),
    ('Sophia Moore', 'BEN777888999', 5500.85),
    ('Lucas Taylor', 'BEN112233445', 2900.40),
    ('Mia Anderson', 'BEN998877665', 1200.30),
    ('Noah Thomas', 'BEN007007007', 7400.00),
    ('Ava Martinez', 'BEN123789456', 4000.90),
    ('James Garcia', 'BEN654321987', 850.50),
    ('Isabella Rodriguez', 'BEN147258369', 6700.20),
    ('Ethan Martinez', 'BEN369258147', 5300.15),
    ('Charlotte Wilson', 'BEN741852963', 6200.00),
    ('Benjamin Harris', 'BEN963852741', 1200.00),
    ('Amelia Clark', 'BEN258369147', 3400.30),
    ('Alexander Lewis', 'BEN456123789', 7100.25),
    ('Lily Walker', 'BEN963741852', 8900.40),
    ('Jack Young', 'BEN852963741', 10000.00)
]
    

In [127]:
create_table(drawee_bank_table)
create_table(beneficiary_bank_table)


Table 'Drawee_bank' does not exist, creating one
PostgreSQL connection is closed
Table created successfully
PostgreSQL connection is closed
Table 'Beneficiary_bank' does not exist, creating one
PostgreSQL connection is closed
Table created successfully
PostgreSQL connection is closed


In [128]:
insert_data_onto_table(drawee_bank_table, drawer_data)
insert_data_onto_table(beneficiary_bank_table, beneficiary_data)

Data inserted successfully into the {table_name} table.
PostgreSQL connection is closed.
Data inserted successfully into the {table_name} table.
PostgreSQL connection is closed.


In [129]:
def cash_transfer(drawee_account, drawee_name, beneficiary_account, amount):
    try:
        # Connect to PostgreSQL database
        connection = psycopg2.connect(**db_params)  # type: ignore
        cursor = connection.cursor()
        
        # Check drawee's details
        cursor.execute(f"""
            SELECT name, balance FROM {drawee_bank_table}
            WHERE account_number = %s;
        """, (drawee_account,))
        drawee = cursor.fetchone()
        
        if not drawee:
            print("Drawee account not found.")
            return
        
        if drawee[0] != drawee_name:
            print("Drawee name does not match.")
            return
        
        drawee_balance = drawee[1]
        print(f"Drawee's current balance: {drawee_balance:.2f}")
        
        if drawee_balance < amount:
            print("Insufficient funds in drawee's account.")
            return
        
        # Calculate new balances
        new_drawee_balance = drawee_balance - amount
        
        # Update drawee's balance
        cursor.execute(f"""
            UPDATE {drawee_bank_table}
            SET balance = %s
            WHERE account_number = %s;
        """, (new_drawee_balance, drawee_account))
        
        # Commit the transaction
        connection.commit()
        
        # Print balance before and after the transaction
        print(f"Cash given to {beneficiary_account}.")
        print(f"Drawee's balance before transaction: {drawee_balance:.2f}")
        print(f"Drawee's balance after transaction: {new_drawee_balance:.2f}")
    
    except (Exception, Error) as error:
        # Rollback the transaction in case of an error
        if connection:
            connection.rollback()
        print("Error during fund transfer:", error)
    
    finally:
        # Ensure the cursor and connection are properly closed
        cursor.close()
        connection.close()
        print("Transaction completed.")

In [130]:
def bank_transfer(drawee_account, drawee_name, beneficiary_name, beneficiary_account, amount):
    try:
        # Connect to PostgreSQL database
        connection = psycopg2.connect(**db_params)  # type: ignore
        cursor = connection.cursor()
        
        # Fetch drawee's details
        cursor.execute(f"""
            SELECT name, account_number, balance FROM {drawee_bank_table}
            WHERE account_number = %s;
        """, (drawee_account,))
        drawee = cursor.fetchone()
        
        if not drawee:
            print("Drawee account not found.")
            return
        
        if drawee[0] != drawee_name:
            print("Drawee name does not match.")
            return
        
        drawee_balance = drawee[2]
        print(f"Drawee's balance before transaction: {drawee_balance:.2f}")
        
        if drawee_balance < amount:
            print("Insufficient funds in drawee's account.")
            return
        
        # Fetch beneficiary's details
        cursor.execute(f"""
            SELECT name, account_number, balance FROM {drawee_bank_table}
            WHERE account_number = %s;
        """, (beneficiary_account,))
        beneficiary = cursor.fetchone()
        
        if not beneficiary:
            print("Beneficiary account not found.")
            return
        
        if beneficiary[0] != beneficiary_name:
            print("Beneficiary name does not match.")
            return
        
        if beneficiary[1] != beneficiary_account:
            print("Beneficiary account number does not match.")
            return
        
        beneficiary_balance = beneficiary[2]
        print(f"Beneficiary's balance before transaction: {beneficiary_balance:.2f}")
        
        # Calculate new balances
        new_drawee_balance = drawee_balance - amount
        new_beneficiary_balance = beneficiary_balance + amount
        
        # Update drawee's balance
        cursor.execute(f"""
            UPDATE {drawee_bank_table}
            SET balance = %s
            WHERE account_number = %s;
        """, (new_drawee_balance, drawee_account))
        
        # Update beneficiary's balance
        cursor.execute(f"""
            UPDATE {drawee_bank_table}
            SET balance = %s
            WHERE account_number = %s;
        """, (new_beneficiary_balance, beneficiary_account))
        
        # Commit the transaction
        connection.commit()
        
        # Print balance after the transaction
        print(f"Funds successfully transferred from account {drawee_name} to account {beneficiary_name}.")
        print(f"Drawee's balance after transaction: {new_drawee_balance:.2f}")
        print(f"Beneficiary's balance after transaction: {new_beneficiary_balance:.2f}")
    
    except (Exception, Error) as error:
        # Rollback the transaction in case of an error
        if connection:
            connection.rollback()
        print("Error during bank transfer:", error)
    
    finally:
        # Ensure the cursor and connection are properly closed
        if cursor:
            cursor.close()
        if connection:
            connection.close()
        print("Transaction completed.")

In [131]:
def EFT_method(drawee_name, drawee_account, beneficiary_name, beneficiary_account, amount):
    try:
        # Connect to PostgreSQL database
        connection = psycopg2.connect(**db_params)  # type: ignore
        cursor = connection.cursor()
        
        # Fetch drawee's details
        cursor.execute(f"""
            SELECT name, account_number, balance FROM {drawee_bank_table}
            WHERE account_number = %s;
        """, (drawee_account,))
        drawee = cursor.fetchone()

        if not drawee:
            print("Drawee account not found.")
            return
        
        if drawee[0] != drawee_name:
            print("Drawee name does not match.")
            return
        
        drawee_balance = drawee[2]
        print(f"Drawee's balance before transaction: {drawee_balance:.2f}")
        
        if drawee_balance < amount:
            print("Insufficient funds in drawee's account.")
            return
        
        # Fetch beneficiary's details
        cursor.execute(f"""
            SELECT name, account_number, balance FROM {beneficiary_bank_table}
            WHERE account_number = %s;
        """, (beneficiary_account,))
        beneficiary = cursor.fetchone()
        
        if not beneficiary:
            print("Beneficiary account not found.")
            return
        
        if beneficiary[0] != beneficiary_name:
            print("Beneficiary name does not match.")
            return
        
        if beneficiary[1] != beneficiary_account:
            print("Beneficiary account number does not match.")
            return
        
        beneficiary_balance = beneficiary[2]
        print(f"Beneficiary's balance before transaction: {beneficiary_balance:.2f}")
        
        # Calculate new balances
        new_drawee_balance = drawee_balance - amount
        new_beneficiary_balance = beneficiary_balance + amount
        
        # Update drawee's balance
        cursor.execute(f"""
            UPDATE {drawee_bank_table}
            SET balance = %s
            WHERE account_number = %s;
        """, (new_drawee_balance, drawee_account))
        
        # Update beneficiary's balance
        cursor.execute(f"""
            UPDATE {beneficiary_bank_table}
            SET balance = %s
            WHERE account_number = %s;
        """, (new_beneficiary_balance, beneficiary_account))
        
        # Commit the transaction
        connection.commit()
        
        # Print balance after the transaction
        print(f"Funds successfully transferred from account {drawee_name} to account {beneficiary_name}.")
        print(f"Drawee's balance after transaction: {new_drawee_balance:.2f}")
        print(f"Beneficiary's balance after transaction: {new_beneficiary_balance:.2f}")
    
    except (Exception, Error) as error:
        # Rollback the transaction in case of an error
        if connection:
            connection.rollback()
        print("Error during bank transfer:", error)
    
    finally:
        # Ensure the cursor and connection are properly closed
        if cursor:
            cursor.close()
        if connection:
            connection.close()
        print("Transaction completed.")

In [132]:
# Sample input for cash transfer, data exttracted using OCR
beneficiary_name = "John Doe"

drawer_name = "Alice Johnson"
account_number_drawer = "ACC123456789"
amount = 100


cash_transfer(account_number_drawer, drawer_name, beneficiary_name, amount)

Drawee's current balance: 1200.50
Cash given to John Doe.
Drawee's balance before transaction: 1200.50
Drawee's balance after transaction: 1100.50
Transaction completed.


In [133]:
# Sample input where the drawer name is incorrect for cash transfer, data exttracted using OCR
beneficiary_name = "John Doe"

drawer_name = "Alicea Johnson"                        # we have changed the drawer name
account_number_drawer = "ACC123456789"
amount = 500


cash_transfer(account_number_drawer, drawer_name, beneficiary_name, amount)

Drawee name does not match.
Transaction completed.


In [134]:
# sample input data for bank transfer, within the bank, data exttracted using OCR
beneficiary_name = "Diana Prince"
beneficiary_account_number = "ACC111222333"

drawer_name = "Bob Smith"
account_number_drawer = "ACC987654321"
amount = 500

bank_transfer(account_number_drawer, drawer_name, beneficiary_name, beneficiary_account_number, amount)


Drawee's balance before transaction: 2450.75
Beneficiary's balance before transaction: 750.30
Funds successfully transferred from account Bob Smith to account Diana Prince.
Drawee's balance after transaction: 1950.75
Beneficiary's balance after transaction: 1250.30
Transaction completed.


In [135]:
# sample incorrect input data for bank transfer, within the bank, data exttracted using OCR
beneficiary_name = "Diana Prince"
beneficiary_account_number = "ACC111222333"

drawer_name = "Bobi Smith"                              # we have changed the name
account_number_drawer = "ACC987654321"
amount = 500

bank_transfer(account_number_drawer, drawer_name, beneficiary_name, beneficiary_account_number, amount)


Drawee name does not match.
Transaction completed.


In [136]:
# sample data for EFT method, data exttracted using OCR
beneficiary_name = "Mia Anderson"
beneficiary_account_number = "BEN998877665"

drawer_name = "Oliver Queen"
account_number_drawer = "ACC741852963"
amount = 500

EFT_method(drawer_name, account_number_drawer, beneficiary_name, beneficiary_account_number, amount)

Drawee's balance before transaction: 5600.25
Beneficiary's balance before transaction: 1200.30
Funds successfully transferred from account Oliver Queen to account Mia Anderson.
Drawee's balance after transaction: 5100.25
Beneficiary's balance after transaction: 1700.30
Transaction completed.


In [137]:
# sample data for EFT method, data exttracted using OCR
beneficiary_name = "Mika Anderson"                              # we have changed the name
beneficiary_account_number = "BEN998877665"

drawer_name = "Oliver Queen"
account_number_drawer = "ACC741852963"
amount = 500

EFT_method(drawer_name, account_number_drawer, beneficiary_name, beneficiary_account_number, amount)

Drawee's balance before transaction: 5100.25
Beneficiary name does not match.
Transaction completed.
